# Making a Geodataframe from Fire location data

In [1]:
#Import the necessary Python moduless
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
from folium import IFrame
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps

/opt/conda/lib/python3.6/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [2]:
dba = pd.read_csv('../data/vector/fire_archive_V1_2871.csv')

def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    return s.map({date:pd.to_datetime(date) for date in s.unique()})

dba.loc[:,'date']=lookup(dba['acq_date'])

dba['monthday'] = dba['date'].dt.strftime('%j')
dba['fileformate'] = dba['date'].dt.strftime('%Y_%m_%d')
dba['year'] = dba['date'].dt.strftime('%Y')


yearlist=dba.drop_duplicates('year')
yearlist1=yearlist['year'].tolist()

dba

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,date,monthday,fileformate,year
0,30.94264,64.14420,305.9,0.39,0.59,2017-12-01,2213,N,VIIRS,n,1,275.3,1.2,NaN,2017-12-01,335,2017_12_01,2017
1,33.76704,70.33032,315.2,0.72,0.75,2017-12-01,2212,N,VIIRS,n,1,277.7,3.2,NaN,2017-12-01,335,2017_12_01,2017
2,38.79974,64.64509,320.2,0.54,0.51,2017-12-01,2211,N,VIIRS,n,1,272.0,2.3,NaN,2017-12-01,335,2017_12_01,2017
3,12.98457,74.85178,305.5,0.39,0.36,2017-12-01,2037,N,VIIRS,n,1,284.4,1.5,NaN,2017-12-01,335,2017_12_01,2017
4,13.16106,80.28531,337.2,0.47,0.48,2017-12-01,2037,N,VIIRS,n,1,278.0,4.9,NaN,2017-12-01,335,2017_12_01,2017
5,14.62569,75.83141,303.8,0.40,0.37,2017-12-01,2037,N,VIIRS,n,1,286.3,0.5,NaN,2017-12-01,335,2017_12_01,2017
6,15.09222,75.91351,313.7,0.40,0.37,2017-12-01,2037,N,VIIRS,n,1,285.7,1.4,NaN,2017-12-01,335,2017_12_01,2017
7,15.16880,76.37869,302.3,0.42,0.37,2017-12-01,2037,N,VIIRS,n,1,282.4,0.7,NaN,2017-12-01,335,2017_12_01,2017
8,15.13774,76.71426,301.6,0.43,0.38,2017-12-01,2037,N,VIIRS,n,1,290.0,0.6,NaN,2017-12-01,335,2017_12_01,2017
9,15.13840,76.71032,305.2,0.43,0.38,2017-12-01,2037,N,VIIRS,n,1,290.1,0.6,NaN,2017-12-01,335,2017_12_01,2017


In [3]:
#First create a GeoSeries of crime locations by converting coordinates to Shapely geometry objects
#Specify the coordinate system ESPG4326 which represents the standard WGS84 coordinate system
fire_geo = gpd.GeoSeries(dba.apply(lambda z: Point(z['longitude'], z['latitude']), 1),crs={'init': 'epsg:4326'})

#Create a geodataframe from the pandas dataframe and the geoseries of shapely geometry objects
firepts = gpd.GeoDataFrame(dba.drop(['longitude', 'latitude'], 1), geometry=fire_geo)
print(firepts.head())

   bright_ti4  scan  track    acq_date  acq_time satellite instrument  \
0       305.9  0.39   0.59  2017-12-01      2213         N      VIIRS   
1       315.2  0.72   0.75  2017-12-01      2212         N      VIIRS   
2       320.2  0.54   0.51  2017-12-01      2211         N      VIIRS   
3       305.5  0.39   0.36  2017-12-01      2037         N      VIIRS   
4       337.2  0.47   0.48  2017-12-01      2037         N      VIIRS   

  confidence  version  bright_ti5  frp  daynight       date monthday  \
0          n        1       275.3  1.2       NaN 2017-12-01      335   
1          n        1       277.7  3.2       NaN 2017-12-01      335   
2          n        1       272.0  2.3       NaN 2017-12-01      335   
3          n        1       284.4  1.5       NaN 2017-12-01      335   
4          n        1       278.0  4.9       NaN 2017-12-01      335   

  fileformate  year                            geometry  
0  2017_12_01  2017            POINT (64.1442 30.94264)  
1  2017_12_0

# calculate state level density of fire observation

In [ ]:
#Read tracts shapefile into GeoDataFrame
state_poly = gpd.read_file('India_states.shp')

#Generate Counts of Assaults per Census Tract
#Spatially join census tracts to assaults (after projecting) and then group by Tract FIPS while counting the number of crimes
point_counts = gpd.tools.sjoin(firepts.to_crs(state_poly.crs), state_poly.reset_index()).groupby('censuscode').size()

#Calculate Assault Density, converting square meters to square miles.
#state_poly['firePSqm'] = (point_counts/(state_poly.geometry.area)).fillna(0)
#state_poly = state_poly.reset_index()
#print(state_poly.head())
point_counts=pd.DataFrame(point_counts)
point_counts['censuscode']=point_counts.index
point_counts.columns=['fireCountPSqm','censuscode']

points_poly=pd.merge(state_poly,point_counts,on='censuscode')
points_poly

# Adding chloropleth map layer

In [ ]:
#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
fire_map = folium.Map([19.17, 72.98], zoom_start = 12)

def add_choropleth(mapobj, gdf, id_field, value_field, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 5, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale)
    return mapobj

#Update basemap with choropleth
fire_map =add_choropleth(fire_map,points_poly, 'censuscode','fireCountPSqm')

# Adding point layer

In [ ]:
def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Create a string of HTML code used in the IFrame popup
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join([row[field] for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(IFrame(label, width = 300, height = 100))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_children(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_children(pt_lyr)
    return mapobj

#Update choropleth with point clusters
fire_map = add_point_clusters(fire_map,firepts , ['instrument'])

In [ ]:
folium.LayerControl().add_to(fire_map) #Add layer control to toggle on/off
#crime_map.save('sf_assaults.html') #save HTML
fire_map #display map